# 1. DB 연결

In [3]:
import pandas as pd
from sqlalchemy import create_engine


In [ ]:

q = '''
select 
from user
limit 10'''
df = pd.read_sql(q,con=mysql)

# 2. 서비스 데이터베이스에서 상품정보 수집

In [11]:
query = f"""SELECT
                    p.id
                  , p.uid
                  , p.status AS status
                  , p.price
                  , pd.description
                  , p.flag_used AS used
                  , p.flag_taekpo AS free_shipping
                  , p.flag_exchg AS exchange
                  , p.category_id
                  , p.enc_location
                  , p.enc_latitude
                  , p.enc_longitude
                  , p.name
                  , p.create_date
                  , p.m_time
                  , p.s3_storage
                  , p.keyword AS keywords
                  , c.name AS category_name
                  , IFNULL(pu.register_date, now()) AS update_time
                  , ps.extended_spec
                  , IFNULL(pe.interest, 0) AS view_count
                  , IFNULL(pe.pfavcnt, 0) AS favorite_count
                  , pe.comment_cnt AS comment_count
                  , pe.is_adult AS adult
                  , pe.ptranscnt AS contact_count
                  , IF(cs.is_removed = 0, 1, 0) AS checkout
                  , IFNULL(ub.bizlicense, 0) AS by_bizseller
                  , p.modified_at
                  , ps.updated AS modified_at_ps
                  , cs.modified_at AS modified_at_cs
                  , su.status AS super_up_status
                  , IF(su.status=2 AND su.type = 1, (SELECT sul.enable FROM ad_super_up_lite_group sul WHERE sul.suid = su.suid), IF(su.status=2, 1, 0)) is_display_super_up
                  , IFNULL(suspd.idx, 0)*IFNULL(sus.sus_id, 0)*IFNULL(suspd.on_off, 0) as is_super_up_shop
                  , IFNULL(suspd.on_off, -1) as is_displayable_super_up_shop
                  , IFNULL(es365.product_reported_count, 0) product_reported_count_365
                  , IFNULL(es365.shop_reported_count, 0) shop_reported_count_365
                  , IFNULL(es365.review_reported_count + es365.review_reply_reported_count, 0) review_reported_count_365
                  , IFNULL(es365.comment_reported_count, 0) comment_reported_count_365
                  , IFNULL(es365.group_post_reported_count, 0) group_post_reported_count_365
                  , IFNULL(es90.product_reported_count, 0) product_reported_count_90
                  , IFNULL(es90.shop_reported_count, 0) shop_reported_count_90
                  , IFNULL(es90.review_reported_count + es90.review_reply_reported_count, 0) review_reported_count_90
                  , IFNULL(es90.comment_reported_count, 0) comment_reported_count_90
                  , IFNULL(es90.group_post_reported_count, 0) group_post_reported_count_90
                  , IFNULL(es30.product_reported_count, 0) product_reported_count_30
                  , IFNULL(es30.shop_reported_count, 0) shop_reported_count_30
                  , IFNULL(es30.review_reported_count + es30.review_reply_reported_count, 0) review_reported_count_30
                  , IFNULL(es30.comment_reported_count, 0) comment_reported_count_30
                  , IFNULL(es30.group_post_reported_count, 0) group_post_reported_count_30
                  , outlink.outlink_url
                  , IFNULL(pc.contact_hope, 0) contact
                  , IFNULL(pn.display_on_neighborhood, 0) as neighborhood
FROM
                    product_info p
                    LEFT JOIN product_up pu ON p.id = pu.pid
                    LEFT JOIN product_spec_json ps ON p.id = ps.pid
                    LEFT JOIN product_description pd ON p.id = pd.pid
                    LEFT JOIN product_ext pe ON p.id = pe.pid
                    LEFT JOIN checkout_status cs ON p.id = cs.pid
                    LEFT JOIN user_bizlicense ub ON p.uid = ub.uid
                    LEFT JOIN product_outlink outlink ON p.id = outlink.pid
                    LEFT JOIN product_contact_option pc ON p.id = pc.pid
                    LEFT JOIN ad_super_up su ON p.id = su.pid AND su.status = 2 AND EXISTS (SELECT 1 FROM ad_super_up_point spu WHERE su.suid = spu.suid AND now() BETWEEN start_at AND end_at)
                    LEFT JOIN ad_super_up_shop sus ON p.uid = sus.uid AND now() BETWEEN sus.start_at AND sus.end_at AND sus.status=2
                    LEFT JOIN ad_super_up_shop_products suspd on p.id=suspd.pid and sus.sus_id=suspd.sus_id
                    LEFT JOIN emergency_uniq_stats es30 ON p.uid = es30.uid AND es30.period = 30
                    LEFT JOIN emergency_uniq_stats es90 ON p.uid = es90.uid AND es90.period = 90
                    LEFT JOIN emergency_uniq_stats es365 ON p.uid = es365.uid AND es365.period = 365
                    LEFT JOIN product_neighborhood pn ON p.id = pn.pid
                    LEFT JOIN categories c ON c.category = p.category_id
WHERE p.create_date BETWEEN '2020-05-28 00:00:00' AND '2020-05-31 23:59:59';"""

df = pd.read_sql(query, con=db_slave)

/Users/emilyPark/.pyenv/versions/3.7.5/envs/bun-analytics/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1292, "Truncated incorrect DOUBLE value: 'ALL'")
  result = self._query(query)


In [12]:
df.to_csv('SA_scoring_df.csv', index=False)

In [18]:
product_template = {
  "category": {
    "name": "남성의류",
    "level1": "010",
    "level2": "111",
    "level3": "222"
  },
  "createdAt": "20200602162425",
  "description": "테스트 물품",
  "modifiedAt": "20200602162425",
  "id": 1232323,
  "keywords": "테스트 태그",
  "location": {
    "name": "경기도 의왕시 백운로46"
  },
  "m_time": 323123213,
  "meta": {
    "adult": 1,
    "contact": 0,
    "display": "neiborhood",
    "exchange": 1,
    "freeShipping": 1,
    "used": 1
  },
  "metrics": {
    "commentCount": 10,
    "contactCount": 20,
    "favoriteCount": 30,
    "viewCount": 40
  },
  "name": "아이폰7",
  "price": 100000,
  "status": 0,
  "updatedAt": "20200602162425",
  "user": {
    "uid": 5397308
  }
}

In [19]:
df.columns

Index(['id', 'uid', 'status', 'price', 'description', 'used', 'free_shipping',
       'exchange', 'category_id', 'enc_location', 'enc_latitude',
       'enc_longitude', 'name', 'create_date', 'm_time', 's3_storage',
       'keywords', 'category_name', 'update_time', 'extended_spec',
       'view_count', 'favorite_count', 'comment_count', 'adult',
       'contact_count', 'checkout', 'by_bizseller', 'modified_at',
       'modified_at_ps', 'modified_at_cs', 'super_up_status',
       'is_display_super_up', 'is_super_up_shop',
       'is_displayable_super_up_shop', 'product_reported_count_365',
       'shop_reported_count_365', 'review_reported_count_365',
       'comment_reported_count_365', 'group_post_reported_count_365',
       'product_reported_count_90', 'shop_reported_count_90',
       'review_reported_count_90', 'comment_reported_count_90',
       'group_post_reported_count_90', 'product_reported_count_30',
       'shop_reported_count_30', 'review_reported_count_30',
       'comment

# 3. ES 벌크 데이터로 변환후 삽입

In [20]:
from datetime import datetime
from time import sleep
import requests
import json

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

In [21]:
def make_header(index, _id):
    return '{"index": {"_index": "%s", "_id": "%s"}}\n' % (index, _id)

In [22]:
es_bulk_data = str()
index_name = "product"
for idx, row in df.iterrows():
    product_template['category']['name'] = row['category_name']
    product_template['category']['level1'] = str(row['category_id'])[:3]
    product_template['category']['level2'] = str(row['category_id'])[3:6]
    product_template['category']['level3'] = str(row['category_id'])[-3:]
    
    product_template['createdAt'] = datetime.strftime(row['create_date'], '%Y%m%d%H%M%S')
    product_template['description'] = ""
    product_template['modifiedAt'] = datetime.strftime(row['modified_at'], '%Y%m%d%H%M%S')
    product_template['id'] = row['id']
    product_template['keywords'] = row['keywords']
    
    product_template['location']['name'] = "경기도 의왕시 백운중앙로 46"
    
    product_template['m_time'] = row['m_time']

    product_template['meta']['adult'] = row['adult']
    product_template['meta']['contact'] = row['contact']
    product_template['meta']['display'] = "None"
    product_template['meta']['exchange'] = row['exchange']
    product_template['meta']['freeShipping'] = row['free_shipping']
    product_template['meta']['used'] = row['used']
    
    product_template['metrics']['commentCount'] = row['comment_count']
    product_template['metrics']['contactCount'] = row['contact_count']
    product_template['metrics']['favoriteCount'] = row['favorite_count']
    product_template['metrics']['viewCount'] = row['view_count']
    
    product_template['name'] = row['name']
    product_template['price'] = row['price']
    product_template['status'] = row['status']
    product_template['updatedAt'] = datetime.strftime(row['update_time'], '%Y%m%d%H%M%S')
    
    product_template['user']['uid'] = row['uid']
    
    query_headers = make_header(index_name, row['id'])
    es_bulk_data += query_headers + json.dumps(product_template) + "\n"
    
    if idx % 1000 == 0:
        requests.post(url="http://127.0.0.1:9200/product/_bulk", 
                      data=es_bulk_data, 
                      headers=headers)
        es_bulk_data = str()
        sleep(10)